In [ ]:
#import libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve,ShuffleSplit
from sklearn.metrics import confusion_matrix,classification_report,roc_curve,roc_auc_score,auc,accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


<h2><b>Load the data</b></h2>

In [ ]:
df=pd.read_csv("../input/heart-disease-uci/heart.csv")
df.head()

In [ ]:
df.isnull().sum()

<p>So, we have no null values in our dataset. Thats promising</p>

In [ ]:
df.dtypes

<h2><b>Exploratory data analysis and Visualization</b></h2>

<p>First of all let's visualize the age distribution in our data</p>

In [ ]:
sns.set_style("whitegrid")
sns.boxplot(x="age",data=df)
plt.title("Box plot showing distribution of age in the dataset")
plt.xlabel("Age in years")


In [ ]:
#lets see the descriptive stats for this box-plot
IQR=stats.iqr(df["age"],interpolation="midpoint")
IQR

<p> Observations that can be made from the figure :<br/>
<ul>
    <li>The median age is close to 55 years.</li>
    <li>25 % of the people are less than 48 years of age.</li>
    <li>25% of the people are above 63 years of age.</li>
    <li>There are no visible outliers in the dataset, although 1.5*IQR may beg to differ.</li>
    <li>Interquartile range is about 13.5</li>
    <li>Looks like about 50 percent of the data is about upper medium to early old age.It's kinda true as generally that's the average life expectancy for most of the adults.</li>
</ul>
</p>

### **Let's visualize if a particular gender is more prone to getting a heart disease**

In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="sex",data=df)
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("Frequency",fontsize=14)


<p>So,on an average females are more prone to heart diseases</p>
<p>So, gender might be a good predictor for heart disease. We need to dive deeper in order to reach a valid conclusion.</p>

<p>Now, let's see how much is this thalassemia reponsible for heart disease </p>
<p>According to google <i>"Thalassemia is an inherited blood disorder characterised by less oxygen-carrying protein (haemoglobin) and fewer red blood cells in the body than normal."</i>. Or in layman terms, with thalassemia rbc's are destroyed at a rapid rate, which leads to fatigue and maybe heart disease. Who knows? Let's find out.</p>

<p> Well, I found some important points regarding the so called thal variable in this dataset. Check it out so that we can infer better from the graph.
    <ul>
        <li>0 maps to null in the original dataset.</li>
        <li>1 maps to 6 in the original dataset. This means that a fixed defect was found.</li>
        <li> 2 maps to 3 in the original dataset. This means that the blood flow was normal.</li>
        <li> 3 maps to 7 in the original dataset. This means that a reversible defect was found.</li>
</ul>
        </p>

<p>So what do they even mean ??
<ul>
    <li>Fixed Defect(1) means that no blood flows in some part of the heart.</li>
    <li>Reversible Defect(2) means that an abnormal blood flow pattern is observed.</li>
</ul>
</p>

In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="thal",data=df)
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p><b>Conclusions:</b>
<ul>
    <li>People suffering from heart disease generally have a type-2 thalassemia. i.e abnormal blood flow pattern is observed.</li>
    <li>Healthy people generally have a normal blood flow.</li>
    <li>Fixed Defect thalassemia is present in both. But there aren't enough samples to make an absolute decision.</li>
    <li>Looks like fixed defect thalassemia is not such a good indicator of whether a person is likely to suffer from a heart disease or not. Will be removed during one hot encoding</li>
    <b> Well abnormal blood flow in heart can cause a shock or..... an attack !!</b>
</ul> 
    </p>

<p> <b>Now , lets see how the chestpain affects the chances of getting a heart disease</b></p>

In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="cp",data=df,palette="Accent")
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p><b>Conclusions:</b><br/>
    <ul>
        <li>1: typical angina:more in people with heart disease.</li>
        <li>2: atypical angina : more in people with heart disease.</li>
        <li>3: non-anginal pain : not conclusive</li>
        <li>0: asymptomatic : people with type 0 cp are not that likely to have a heart disease.</li>

  </ul>
</p>

<p> <b>Understanding the pattern for exercise induced angina</b></p>

In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="exang",data=df,palette="Set2")
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p>
<b>Conclusions:</b><ul>
    <li>
    So, most people who had a heart disease, didn't have exercise induced angina/pain.
    </li>
    <li>
        People who didn't suffer from heart disease had approximately the same probability of suffering from exang and not suffering from it.
    </li>
    </ul>
</p>

<p><b>FBS</b></p>
</p>It signifies The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
Greater than 120mg/dl means suffering from diabetes.</p>

In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="fbs",data=df,palette="Set1")
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.show()

<p> Nothing conclusive can be said. The number of high fbs and low fbs are looking very close in both the cases</p>

<p>Time to look at the trend for restecg.<br/><b>restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)</b></p>

In [ ]:
copy_df=df.copy()


In [ ]:
copy_df['restecg'][copy_df['restecg']==0]='normal'
copy_df['restecg'][copy_df['restecg']==1]="ST-T wave abnormality"
copy_df['restecg'][copy_df['restecg']==2]="left ventricular hypertrophy"
copy_df['slope'].value_counts()

In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="restecg",data=copy_df)
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("frequency",fontsize=14)
plt.show()

<p><b>Conclusions:</b>
    <ul>
    <li>ST-T wave abnormality is predominant in people with heart disease.Which is true considering that, ST-T wave abnormality is predominant in people with heart disease. </li>
        <li>Not enough samples for left ventricular hypertrophy. So it cannot be deciding factor to predict heart disease as sample size won't allow it to be a major predictor in classification algorithm.</li>
        <li>Normal ecg test scores are higher in people who not suffering from heart disease, but not by a very satisfactory margin.</li>
    </ul>
    </p>

<p> Now let's understand what slope is.In this dataset it is the slope of the peak exercise ST segment.Exercise tolerance testing is an important diagnostic and prognostic tool for assessing patients with suspected or known ischaemic heart disease. During exercise, coronary blood flow must increase to meet the higher metabolic demands of the myocardium. Limiting the coronary blood flow may result in electrocardiographic changes.ST segment depression (horizontal or downsloping) is the most reliable indicator of exercise-induced ischaemia.  </p>
<p>Value 1: upsloping, Value 2: flat, Value 3: downsloping)</p>

In [ ]:
copy_df['slope'][copy_df['slope']==0]='downsloping'
copy_df['slope'][copy_df['slope']==2]="flat"
copy_df['slope'][copy_df['slope']==1]="upsloping"


In [ ]:
plt.figure(figsize=(9,9))
sns.countplot(x="target",hue="slope",data=copy_df,palette="Paired_r")
plt.xlabel("Heart Disease",fontsize=14)
plt.ylabel("frequency",fontsize=14)
plt.show()

<p><b>Conclusions:</b>
    <ul>
        <li>Upsloping in st segment means normal electrocardiographic changes during exercise. Which is good. And from the plot, healthy people generally have upsloping st segments.</li>
        <li>Downsloping means abnormal electrocardiographic changes, which is bad. And people suffering from heart diseases generally give such readings.</li>
</ul>
    </p>

<p>So, now we are done with basic categorical understanding. Let's get started with the quantitative variable analysis</p>

In [ ]:
df.head()

<h2><b>Let's see which age group and gender have more cholestrol levels</b></h2>

In [ ]:
copy_df['sex'][copy_df['sex']==0]="female"
copy_df['sex'][copy_df["sex"]==1]="male"

In [ ]:

plt.figure(figsize=(9,9))
sns.regplot(x="age",y="chol",data=copy_df)
sns.scatterplot(x="age",y="chol",data=copy_df,hue="sex",palette="flare")
plt.xlabel("Age",fontsize=14)
plt.ylabel("Cholestrol",fontsize=14)
plt.show()

<p>So, there is a weak positive correlation between age and cholestrol.(essentially no correlation)</p>
<p>Some old women have unusually high cholestrol levels</p>

In [ ]:
figure,axs=plt.subplots(1,2,figsize=(8,8))
sns.boxplot(x="sex",y="chol",data=copy_df,palette="Set1",ax=axs[0])
sns.boxplot(x="sex",y="age",data=copy_df,palette="flare_r",ax=axs[1])
axs[0].set_xlabel("Sex",fontsize=14)
axs[0].set_xlabel("Cholestrol",fontsize=14)
axs[1].set_xlabel("Sex",fontsize=14)
axs[1].set_xlabel("Age",fontsize=14)
plt.show()

<p>Well! We were right. Some outliers are present</p>
<p>From the distribution, it is evident that on an average, females have higher cholestrol levels than male.</p>
<p> 50% females in the dataset have cholestrol between 220-300. While half the male people in the dataset have have cholestrol levels between 210-260</p>
<p>50 % of the males in the dataset are between 48-60 years of age whilst on the other hand 50% of the females are between 49-65 years of age</p>

<h2>Swarm plot for various categorical data and heart disease</h2>

In [ ]:
figure,axs=plt.subplots(2,2,figsize=(10,10))
y_labels=['trestbps','age','thalach','oldpeak']
index=0
for i,j in zip([0,0,1,1],[0,1,0,1]):
    sns.stripplot(x="target",y=y_labels[index],hue="sex",data=copy_df,ax=axs[i][j])
    axs[i][j].set_xlabel("heart disease",fontsize=14)
    axs[i][j].set_ylabel(y_labels[index],fontsize=14)
    index+=1
    

plt.show()

<p>1. So, people with higher resting blood pressure don't necessarily have heart disease</p>
<p>2. People with maximum possible heart rates(thalach) above 140 generally suffer from a heart disease</p>
<p>3. People with lower old peaks are prone to heart diseases.</p>
<p>4. Looks like age doesn't  tell alone if a person suffers from heart disease or not.</p>

In [ ]:
figure,axs=plt.subplots(2,2,figsize=(10,10))
y_labels=['trestbps','thalach','oldpeak',"chol"]
index=0
for i,j in zip([0,0,1,1],[0,1,0,1]):
    sns.scatterplot(x="age",y=y_labels[index],hue="target",data=copy_df,ax=axs[i][j],palette="Set1")
    axs[i][j].set_xlabel("age",fontsize=14)
    axs[i][j].set_ylabel(y_labels[index],fontsize=14)
    index+=1
    

plt.show()


<p>1. People below 50 years of age and having low trestbps generally suffer from heart disease. </p>
<p>2. People with lower age and high maximum possible heart rates(thalach) suffer from heart disease.</p>
<p>3. People with lower oldpeak irrespective of the age have heart disease</p>
<p>4. Young people with cholestrol generally have heart disease. Cholestrol increases with age so having high cholestrol levels during young age is bad</p>

<h2>Pairplot for numeric variables</h2>

In [ ]:
numeric_variables=['trestbps','thalach','oldpeak',"chol","age","target"]
sns.pairplot(copy_df[numeric_variables],hue="target",palette="Accent")

<p>1. Lower tbs means heart disease. </p>
<p>2. High thalach means heart disease.</p>
<p>3. Low oldpeak indicates heart disease. </p>
<p>4. Having even small amounts of cholestrol can lead to a heart disease.</p>
<p>5. Age alone doesn't determine heart disease probability.</p>

<h2>Feature Engineering for Model Selection</h2>

In [ ]:
#converting categorical to dummy
hot_encoder=OneHotEncoder()
dummy_columns=["cp","restecg","slope","ca","thal"]
dummy_df=df[dummy_columns]
encoded=hot_encoder.fit_transform(dummy_df).toarray()
features=hot_encoder.get_feature_names(dummy_columns)

dummy_df=pd.DataFrame(encoded,columns=features)
dummy_df.head()

df_new=df.drop(dummy_columns,axis=1)
df_new=pd.concat((dummy_df,df_new),axis=1)
df_new.head()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(data=df_new.corr())
plt.show()

<p>looks like we fell into the dummy variable trap.</p>

In [ ]:
#using pandas to get out of dummy variable trap
dummy_df=pd.get_dummies(df[dummy_columns].astype(str),drop_first=True)
dummy_df.head()

In [ ]:
#constructing a new dataframe
df_new =df.drop(dummy_columns,axis=1)
df_new=pd.concat((dummy_df,df_new),axis=1)
plt.figure(figsize=(12,12))
sns.heatmap(data=df_new.corr())
plt.show()


<p>still some of the variables are highly correlated</p>

In [ ]:
# #we will choose best feature from thal_2,thal_3 and slope_2, slope_1
# features=["thal_2","thal_3","slope_2","slope_1"]
# for feat in features:
#     print(df_new[[feat,"target"]].corr())
    
# #so, thal_2 and slope_2 has higher correlation between target features
# #dropping thal_3 and slope_1
# df_new.drop(labels=["thal_3","slope_1"],inplace=True,axis=1)
# df_new.head()

<h2>Model Training</h2>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y=df_new['target']
X=df_new.drop(labels='target',axis=1)
print(X.shape,y.shape)

#normalizing the data
columns=X.columns
X=pd.DataFrame(scaler.fit_transform(X),columns=columns)


#splitting the data into train and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
X_train.describe().apply(lambda s:s.apply('{0:.5f}'.format))

<h2>SVM</h2>

<p>We will use hyper-parameter tuning to find the best params for the svc classifier</p>

In [ ]:
C=[0.01,0.03,0.1,0.3,1,3,10,30,100]
gamma=[0.01,0.03,0.1,0.3,1,3,10,30,100]
kernel=['rbf','poly','sigmoid']
params={'C':C,'gamma':gamma,'kernel':kernel}

In [ ]:

grid_s=GridSearchCV(SVC(),params,verbose=0)
grid_s.fit(X_train,y_train)
print(grid_s.best_estimator_)
print(grid_s.best_params_)

<p> the best params that I got for svm were</p>
<p>SVC(C=100, gamma=0.01, kernel='sigmoid')</p>

In [ ]:
#fit the model
model=SVC(C=30,gamma=0.03,kernel="sigmoid")
model.fit(X_train,y_train)


In [ ]:
#learning curve plotter function
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:

fig, axes = plt.subplots(3, 1, figsize=(10, 15))
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
plot_learning_curve(model, "SVC(C=100, gamma=0.01, kernel='sigmoid')", X_train, y_train, axes=axes, ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

In [ ]:
#some helper functions
#accuracy score
def score(y_test,predictions):
    print(classification_report(y_test,predictions))
    print(pd.DataFrame(predictions).value_counts())
    print("Accuracy score ={score}".format(score=accuracy_score(y_test,predictions)))

#confusion matrix plotter
def plot_matrix(y_test,predictions):
    matrix=confusion_matrix(y_test,predictions)
    matrix=matrix.transpose()
    cm_df=pd.DataFrame(matrix,index=["Healthy","Heart Disease"],columns=["Healthy","Heart Disease"])
    sns.heatmap(cm_df,annot_kws={"size":16},annot=True,fmt="d")

#roc curve plotter
def plot_roc(y_test,predictions):
    actual_vals=pd.DataFrame(pd.get_dummies(y_test))
    predictions=pd.DataFrame(pd.get_dummies(predictions))
    print('0: {}'.format(predictions[predictions[0]==1][0].sum()))
    print('1 :{}'.format(predictions[predictions[1]==1][1].sum()))
    predictions.head()
    #compute roc curve and roc area for each curve
    fpr=dict()
    tpr=dict()
    roc_auc=dict()
    n_classes=2

    #loop for each class
    for i in range(n_classes):
        fpr[i],tpr[i],_=roc_curve(actual_vals.iloc[:,i],predictions.iloc[:,i])
        roc_auc[i]=auc(fpr[i],tpr[i])
    #micro-average roc curve
    fpr["micro"],tpr["micro"],_=roc_curve(actual_vals.to_numpy().ravel(),predictions.to_numpy().ravel())
    roc_auc["micro"]=auc(fpr["micro"],tpr["micro"])
    
    colors = [ 'darkorange', 'cornflowerblue']
    #main plotter
    lw=2
    plt.figure(figsize=(10,8))
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)
    labels=['Healthy','Heart Disease']
    for i,color in zip(range(n_classes),colors):
         plt.plot(fpr[i], tpr[i], color=color,lw=lw,label='ROC curve of class {0} {name} (area = {area:0.2f})'
                 ''.format(i, name=labels[i],area=roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for heart disease prediction')
    plt.legend(loc="lower right")
    plt.show()
    

In [ ]:
predictions=model.predict(X_test)
score(y_test,predictions)

In [ ]:
#confusion matrix
plot_matrix(y_test,predictions)

In [ ]:
#roc curve
plot_roc(y_test,predictions)

<h2>Decision Tree</h2>

In [ ]:
model=DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
score(y_test,predictions)

<p> Not that good. Performing hyperparameter tuning</p>

In [ ]:
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}


#hyper parameter tuning using grid search
grid_s=GridSearchCV(estimator=DecisionTreeClassifier(),param_grid=params,cv=4,n_jobs=-1,verbose=1,scoring="accuracy")
grid_s.fit(X_train,y_train)

In [ ]:
print(grid_s.best_params_)
print(grid_s.best_estimator_)

In [ ]:
model=DecisionTreeClassifier(max_depth=10, min_samples_leaf=5,criterion="gini")
model.fit(X_train,y_train)

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(20,20))
out= tree.plot_tree(model,
                   feature_names=X_train.columns,
                   class_names=['Healthy', "Heart Disease"],
                   filled=True)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor('red')
        arrow.set_linewidth(3)


<h2>Predictions and accuracy </h2>

In [ ]:
predictions=model.predict(X_test)
score(y_test,predictions)

<h2> Confusion Matrix</h2>

In [ ]:
plot_matrix(y_test,predictions)

<h2>Roc Curve</h2>

In [ ]:
plot_roc(y_test,predictions)

<h2>Logistic Regression</h2>

In [ ]:
model=LogisticRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
score(y_test,predictions)

<h2>Hyperparameter tuning</h2>

In [ ]:
C=[0.01,0.03,0.1,0.3,1,3,10,30,100]
penalty=["l1", "l2", "elasticnet"]

params={'C':C,'penalty':penalty}

#hyper parameter tuning using grid search
grid_s=GridSearchCV(estimator=LogisticRegression(),param_grid=params,cv=4,n_jobs=-1,verbose=1,scoring="accuracy")
grid_s.fit(X_train,y_train)

In [ ]:
print(grid_s.best_estimator_)
print(grid_s.best_params_)


In [ ]:
model=LogisticRegression(C=1,penalty='l2')
model.fit(X_train,y_train)

<h2>Learning Curve</h2>

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 15))
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
plot_learning_curve(model, "LogisticRegression(C=1, penalty=l2)", X_train, y_train, axes=axes, ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

<h2>Predictions and accuracy</h2>

In [ ]:
predictions=model.predict(X_test)
score(y_test,predictions)

<h2>Confusion Matrix</h2>

In [ ]:
plot_matrix(y_test,predictions)

<h2>ROC Curve</h2>

In [ ]:
plot_roc(y_test,predictions)

<h2>Random Forest Classifier</h2>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

<h2>Default Model Test</h2>

In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
score(y_test,predictions)

<h2>Hyperparameter tuning</h2>

In [ ]:
# parameters_dict = {'n_estimators':[3,5,10,20,25,30,40,60,70,100],'max_features':["auto","sqrt","log2"]
#                   ,"max_depth": [2,5,6,10,None],"criterion":["gini","entropy"], "min_samples_split" : [0.1, 0.2, 0.3, 0.4,0.9,2],
#                   "min_samples_leaf" :[0.1, 0.2, 0.3, 0.4,1]}
# grid_s=GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=1),param_grid=parameters_dict,verbose=0)
# grid_s.fit(X_train,y_train)


In [ ]:
print(grid_s.best_estimator_)

In [ ]:
model=RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=20,
                       n_jobs=-1, random_state=1)
model.fit(X_train,y_train)

In [ ]:
predictions=model.predict(X_test)
score(y_test,predictions)

<h2>Confusion Matrix</h2>

In [ ]:
plot_matrix(y_test,predictions)

<h2>Roc curve</h2>

In [ ]:
plot_roc(y_test,predictions)

<h2>Artifical Neural Network</h2>

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
ann=Sequential()
ann.add(Dense(activation="relu",input_dim=22,units=4,kernel_initializer="uniform"))
ann.add(Dense(activation="relu",units=18,kernel_initializer="uniform"))
ann.add(Dense(activation="sigmoid",units=1,kernel_initializer="uniform"))
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
ann.fit(X_train,y_train,batch_size=8,epochs=100)

In [ ]:
preds=ann.predict(X_test)
preds=(preds>0.5)
preds=[int(ele[0])for ele in preds.tolist()]
preds=np.array(preds)


In [ ]:
score(y_test,preds)

<h2>Confusion Matrix</h2>

In [ ]:
plot_matrix(y_test,preds)

<h2>ROC curve</h2>

In [ ]:
plot_roc(y_test,preds)

<p>High precision is necessary for heart disease prediction and we do achieve it.</h2>

<p>It's possible that the model performs better by removing some features, which i'll do in the  future</p>